## Beispiel wie man code aus einem String ausführen kann

In [21]:
code_to_execute = "print('Hello, World!')"

try:
    exec(code_to_execute)
except Exception as e:
    print(f"An error occurred: {e}")

Hello, World!


## Following steps are needed for testing:
1. Identify code from agent output string (ask agent to respond only with code)
2. Save code in dedicated variable 
3. Compare code output with ground trouth output defined by researcher

Before setting up the testing framework like this, we have to know which functions we want to test because this determines the ground truth ouput. Therefore the research for which functions to use should happen before setting up the framework @Felix

See below for sample test walkthrough:

In [32]:
agent_input = "How can I calculate the mean of all the values this list: [1, 2, 3, 4, 5] in Python? Please answer only with the code. Save the result in a variable called 'mean'."

agent_output = """
my_list = [1, 2, 3, 4, 5]
mean = sum(my_list) / len(my_list)
print(mean)
"""

code = agent_output # for this example this is not needed, however if the agent ouput requires extraction this should happen before this statement

desired_result = 3.0 # to be defined by the researcher

def test_my_function(code, desired_result):
    local_vars = {}
    exec(code, globals(), local_vars)
    agent_result = local_vars.get('mean', None)

    if agent_result == desired_result:
        print("Agent output was correct.")
    else:
        print('Agent output was not correct.')

test_my_function(code, desired_result)

3.0
Agent output was correct.
